# Rare-variant aggregate association analysis of white Caucasian individuals from the UK Biobank with age-related hearing impairment (ARHI) traits

This notebook focuses on performing association analysis of rare variants from exome data of white Caucasian individuals from the UK Biobank that have ARHI 

The phenotypes analyzed here are:

1. Hearing aid f.3393 - H-aid
2. Hearing difficulty f.2247 - H-diff
3. Hearing difficulty with background noise f.2257 - H-noise
4. A combined phenotype of f.2247 & f.2257 - H-both

The software used for these analyses was Regenie and the workflows to run the analyses are [here](https://github.com/cumc/bioworkflows/blob/master/GWAS/LMM.ipynb)

The docker image that contains all the necessary software installations to run this notebook and workflow can be found retrieve by the following command:
`docker pull statisticalgenetics/lmm:3.0`

# 10-13-21 Analysis of age-related hearing impairment traits

## Create the anno_file, set_list_file and mask_files necessary for burden test

In [ ]:
burden_dir=~/UKBiobank/results/REGENIE_results/results_burden_exome/101321_burden_files
anno_sbatch_burden=~/UKBiobank/results/REGENIE_results/results_burden_exome/101321_burden_files/ukb23155_200Kexomes_burdenfiles_$(date +"%Y-%m-%d").sbatch
annotated_file_hg38=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz
bim_name=~/UKBiobank/results/ukb23155_200Kexomes_annovar/091321_exome_bim_merge/ukb23155_chr1_chr22_091321.bim
job_size=1
name_prefix='ukb23155_chr1_chr22_qcexomedata_101321_burden_files'
anno_sos=~/project/bioworkflows/variant-annotation/annovar.ipynb
tpl_file=~/project/bioworkflows/admin/csg.yml
container_annovar=$HOME/containers/gatk4-annovar.sif

anno_args="""burden_files
    --cwd $burden_dir
    --annotated_file $annotated_file_hg38
    --bim_name $bim_name
    --name_prefix $name_prefix
    --job_size $job_size
    --container_annovar $container_annovar
    --container_lmm /mnt/mfs/statgen/containers/lmm.sif
"""

sos run ~/working/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $anno_sos \
    --to-script $anno_sbatch_burden\
    --args "$anno_args"


# Global variables

In [ ]:
lmm_burden_dir_regenie=~/UKBiobank/results/REGENIE_results/results_burden_exome
hearing_pheno_path=~/UKBiobank/phenotype_files/hearing_impairment/
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
lmm_job_size=1

# Regenie burden parameters

In [ ]:
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
anno_file=~/UKBiobank/results/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_101321.hg38.hg38_multianno.renamedcols.mask_file
aaf_file=~/UKBiobank/results/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
bsize=1000
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.csv
build_mask=max
aaf_bins='0.005 0.01'
covarCol=sex
qCovarCol='age PC1 PC2'
ylim=0
reverse_log_p=True
numThreads=20

## Discovery sample

### H-aid

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/150k/091321_f3393_hearing_aid
lmm_sbatch_regenie=$lmm_burden_dir_regenie/150k/091321_f3393_hearing_aid/f3393_hearing_aid_150k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_aid_f3393_expandedwhite_78848ind_150K.tsv
phenoCol=f3393

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-diff

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/150k/090321_f2247_hearing_difficulty
lmm_sbatch_regenie=$lmm_burden_dir_regenie/150k/090321_f2247_hearing_difficulty/f2247_hearing_difficulty_150k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_107507ind_150K.tsv
covarFile=$hearing_pheno_path/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_107507ind_150K.tsv
phenoCol=f2247

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-noise

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/150k/090321_f2257_hearing_noise
lmm_sbatch_regenie=$lmm_burden_dir_regenie/150k/090321_f2257_hearing_noise/f2257_hearing_noise_150k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_noise_f2257_expandedwhite_123538ind_150K.tsv
phenoCol=f2257

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-both

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/150k/090321_Combined_f2247_f2257
lmm_sbatch_regenie=$lmm_burden_dir_regenie/150k/090321_Combined_f2247_f2257/Combined_f2247_f2257_150k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Combined_f2247_f2257_expandedwhite_102133ind_150K.tsv
phenoCol=f2247_f2257

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

## Replication sample

### H-aid

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/50k/091321_f3393_hearing_aid
lmm_sbatch_regenie=$lmm_burden_dir_regenie/50k/091321_f3393_hearing_aid/f3393_hearing_aid_50k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_aid_f3393_expandedwhite_24189ind_50K.tsv
phenoCol=f3393
covarCol=sex

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-diff

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/50k/090321_f2247_hearing_difficulty
lmm_sbatch_regenie=$lmm_burden_dir_regenie/50k/090321_f2247_hearing_difficulty/f2247_hearing_difficulty_50k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_34596ind_50K.tsv
phenoCol=f2247

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-noise

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/50k/090321_f2257_hearing_noise
lmm_sbatch_regenie=$lmm_burden_dir_regenie/50k/090321_f2257_hearing_noise/f2257_hearing_noise_50k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_noise_f2257_expandedwhite_38723ind_50K.tsv
phenoCol=f2257
covarCol=sex

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""
sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-both

In [ ]:
lmm_dir_regenie=/mnt/mfs/statgen/UKBiobank/results/REGENIE_results/results_burden_exome/50k/090321_Combined_f2247_f2257
lmm_sbatch_regenie=/mnt/mfs/statgen/UKBiobank/results/REGENIE_results/results_burden_exome/50k/090321_Combined_f2247_f2257/Combined_f2247_f2257_50k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Combined_f2247_f2257_expandedwhite_32878ind_50K.tsv
phenoCol=f2247_f2257

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

## Mega sample
### H-aid

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/101321_f3393_hearing_aid
lmm_sbatch_regenie=$lmm_burden_dir_regenie/101321_f3393_hearing_aid/f3393_hearing_aid_200k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_aid_f3393_expandedwhite_6436cases_96601ctrl_PC1_2.tsv
phenoCol=f3393

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-diff

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/101321_f2247_hearing_difficulty
lmm_sbatch_regenie=$lmm_burden_dir_regenie/101321_f2247_hearing_difficulty/f2247_Hearing_difficulty_200k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_45502cases_96601ctrl_PC1_2.tsv
phenoCol=f2247

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-noise

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/101321_f2257_hearing_noise
lmm_sbatch_regenie=$lmm_burden_dir_regenie/101321_f2257_hearing_noise/f2257_hearing_noise_200k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Hearing_noise_f2257_expandedwhite_65660cases_96601ctrl_PC1_2.tsv
phenoCol=f2257

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

### H-both

In [ ]:
lmm_dir_regenie=$lmm_burden_dir_regenie/101321_Combined_f2247_f2257
lmm_sbatch_regenie=$lmm_burden_dir_regenie/101321_Combined_f2247_f2257/Combined_f2247_f2257_200K_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/090321_UKBB_Combined_f2247_f2257_expandedwhite_38410cases_96601ctrl_PC1_2.tsv
phenoCol=f2247_f2257

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"